In [1]:
# ============================================================
# 🚀 Logo Test Dash App
# ============================================================

from dash import Dash, html

# Create app
app = Dash(__name__)

# Layout with two logos
app.layout = html.Div([
    html.Div([
        html.Img(
            src="/assets/essgi_logo.png",  # First logo path
            style={"height": "110px", "margin": "10px"}
        ),
        html.Img(
            src="/assets/dbu_logo.png",    # Second logo path
            style={"height": "110px", "margin": "10px"}
        )
    ], style={"textAlign": "center", "display": "flex", "justifyContent": "center", "alignItems": "center"}),

    html.H1(
        "✅ Logo Test Successful!",
        style={"textAlign": "center", "color": "#003366", "marginTop": "10px"}
    ),

    html.P(
        "If you see both ESSGI and DBU logos above, your logo paths are working correctly.",
        style={"textAlign": "center", "color": "#555"}
    )
])

if __name__ == "__main__":
    app.run(debug=True, port=8051, use_reloader=False)


In [6]:
# ==========================================================
# 🌍 Total Electron Content (vTEC) Prediction — Addis Ababa
# Developed by Ethiopian Space Science and Geospatial Institute (ESSGI)
# Internship Project by Data Science Students from Debre Birhan University
# ==========================================================

import dash
from dash import dcc, html, Input, Output, State
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import plotly.graph_objs as go
import requests
import lightgbm as lgb
import os

# ------------------------------
# 1️⃣ Load & prepare historical data
# ------------------------------
file_path = r"C:\Users\pc\Desktop\vtec_dashboard\quiet_data_clean_corrected.csv"
quiet_df = pd.read_csv(file_path)

# Cyclical time features
quiet_df['DOY_s'] = np.sin(2 * np.pi * quiet_df['DOY'] / 365)
quiet_df['DOY_c'] = np.cos(2 * np.pi * quiet_df['DOY'] / 365)
quiet_df['HH_s']  = np.sin(2 * np.pi * quiet_df['Hour'] / 24)
quiet_df['HH_c']  = np.cos(2 * np.pi * quiet_df['Hour'] / 24)

FEATURES = ['DOY_s', 'DOY_c', 'HH_s', 'HH_c', 'Sunspot_No', 'Lat', 'Lon', 'f10_7_index']
X = quiet_df[FEATURES]
y = quiet_df['vTEC_corrected']

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

lgb_model = lgb.LGBMRegressor(n_estimators=500, learning_rate=0.05, num_leaves=31, random_state=42)
print("[LightGBM] [Info] Start training from score", np.mean(y_train))
lgb_model.fit(X_train, y_train)

# ------------------------------
# NOAA helpers
# ------------------------------
OBS_URL = "https://services.swpc.noaa.gov/json/solar-cycle/observed-solar-cycle-indices.json"
PRED_URL = "https://services.swpc.noaa.gov/json/solar-cycle/predicted-solar-cycle.json"

def _extract_time_tag(row):
    for k in ('time-tag', 'time_tag', 'timeTag', 'time'):
        if k in row and row[k]:
            return str(row[k]).split('T')[0][:7]
    return None

def _get_value(row, keys):
    for k in keys:
        if k in row and row[k] is not None:
            try:
                return float(row[k])
            except (ValueError, TypeError):
                pass
    return None

def fetch_noaa_solar_vars(year: int, month: int):
    ym = f"{int(year):04d}-{int(month):02d}"
    try:
        obs = requests.get(OBS_URL, timeout=10)
        if obs.ok:
            for row in obs.json():
                if _extract_time_tag(row) == ym:
                    ssn = _get_value(row, ['ssn', 'sunspot_number', 'sunspotno'])
                    f10 = _get_value(row, ['f10.7', 'f107', 'flux', 'f10.7_adj'])
                    if ssn and f10:
                        return ssn, f10
    except:
        pass
    try:
        pred = requests.get(PRED_URL, timeout=10)
        if pred.ok:
            for row in pred.json():
                if _extract_time_tag(row) == ym:
                    ssn = _get_value(row, ['predicted_ssn', 'ssn'])
                    f10 = _get_value(row, ['predicted_f10.7', 'f10.7'])
                    if ssn and f10:
                        return ssn, f10
    except:
        pass
    avg = quiet_df.groupby('DOY')[['Sunspot_No', 'f10_7_index']].mean()
    doy = pd.Timestamp(year=int(year), month=int(month), day=15).dayofyear
    if doy in avg.index:
        row = avg.loc[doy]
        return float(row['Sunspot_No']), float(row['f10_7_index'])
    return 50.0, 100.0

# ------------------------------
# 2️⃣ Dash App Layout
# ------------------------------
app = dash.Dash(__name__)
server = app.server
app.title = "vTEC Prediction — ESSGI"

background_style = {
    "backgroundImage": "url('/assets/space_bg.jpg')",
    "backgroundSize": "cover",
    "backgroundAttachment": "fixed",
    "minHeight": "100vh",
    "padding": "20px"
}

app.layout = html.Div([
    # Header with logos
    html.Div([
        html.Div([
            html.Img(src="/assets/essgi_logo.png",
                     style={"height": "90px", "margin": "auto", "display": "block"})
        ], style={"flex": "1", "textAlign": "center"}),

        html.Div([
            html.H1("🌍 Total Electron Content (vTEC) Prediction — Addis Ababa",
                    style={"color": "#ffffff", "textAlign": "center", "fontSize": "40px",
                           "textShadow": "2px 2px 6px rgba(0,0,0,0.6)"}),
            html.H3("Ethiopian Space Science and Geospatial Institute (ESSGI)",
                    style={"color": "#b3c7ff", "textAlign": "center"}),
            html.P("Internship Project by Data Science Students from Debre Birhan University",
                   style={"textAlign": "center", "fontStyle": "italic", "color": "#cfcfcf"}),
            html.P("👨‍💻 Haile Sintayehu & Esayas Melaku",
                   style={"textAlign": "center", "color": "#a0bfff", "fontWeight": "bold"})
        ], style={"flex": "3"}),

        html.Div([
            html.Img(src="/assets/dbu_logo.png",
                     style={"height": "90px", "margin": "auto", "display": "block"})
        ], style={"flex": "1", "textAlign": "center"})
    ], style={
        "display": "flex", "justifyContent": "space-around", "alignItems": "center",
        "padding": "20px", "backgroundColor": "rgba(0, 20, 60, 0.7)",
        "borderRadius": "12px", "marginBottom": "20px"
    }),

    # Input section
    html.Div([
        html.Label("Year", style={"color": "green", "marginRight": "10px"}),
        dcc.Input(id='input_year', type='number', value=2031,
                  style={"width": "100px", "borderRadius": "8px", "textAlign": "center"}),

        html.Label("Month", style={"color": "yellow", "marginLeft": "20px", "marginRight": "10px"}),
        dcc.Input(id='input_month', type='number', value=1,
                  style={"width": "80px", "borderRadius": "8px", "textAlign": "center"}),

        html.Label("Day", style={"color": "red", "marginLeft": "20px", "marginRight": "10px"}),
        dcc.Input(id='input_day', type='number', value=1,
                  style={"width": "80px", "borderRadius": "8px", "textAlign": "center"}),

        html.Button('🔮 Predict Daily vTEC', id='predict_button',
                    style={"marginLeft": "30px", "padding": "8px 20px",
                           "backgroundColor": "#0052cc", "color": "black",
                           "border": "none", "borderRadius": "8px",
                           "cursor": "pointer"})
    ], style={"textAlign": "center", "margin": "20px"}),

    html.Div(id='noaa_values',
             style={"textAlign": "center", "marginBottom": "10px", "color": "blue"}),

    dcc.Graph(id='daily_pred_plot',
              style={"height": "500px", "backgroundColor": "#ffffff",
                     "borderRadius": "10px"}),

    # Footer
    html.Footer([
        html.P("© 2025 Ethiopian Space Science and Geospatial Institute — All Rights Reserved.",
               style={"textAlign": "center", "fontSize": "12px", "color": "#ccc"}),
        html.P("Developed by Haile Sintayehu & Esayas Melaku — Debre Birhan University.",
               style={"textAlign": "center", "fontSize": "12px", "color": "#ccc"}),
        html.A("Visit Official ESSGI Website", href="https://www.essgi.gov.et", target="_blank",
               style={"display": "block", "textAlign": "center", "marginBottom": "10px",
                      "color": "#4da6ff"})
    ], style={"marginTop": "40px"})
], style=background_style)

# ------------------------------
# 3️⃣ Prediction Callback (Improved)
# ------------------------------
@app.callback(
    Output('daily_pred_plot', 'figure'),
    Output('noaa_values', 'children'),
    Input('predict_button', 'n_clicks'),
    State('input_year', 'value'),
    State('input_month', 'value'),
    State('input_day', 'value')
)
def predict_daily_vtec(n_clicks, year, month, day):
    if not n_clicks:
        return go.Figure(), "Enter a date and click 'Predict Daily vTEC'."

    # Fetch NOAA values
    sunspot, f107 = fetch_noaa_solar_vars(year, month)

    # Validate date
    try:
        date = pd.Timestamp(year, month, day)
    except:
        return go.Figure(), "❌ Invalid date. Please enter a valid year, month, and day."

    days_in_year = 366 if date.is_leap_year else 365
    doy = date.dayofyear
    hours = np.arange(0, 24)
    preds = []

    # Predict vTEC for each hour
    for h in hours:
        row_df = pd.DataFrame({
            'DOY_s': [np.sin(2 * np.pi * doy / days_in_year)],
            'DOY_c': [np.cos(2 * np.pi * doy / days_in_year)],
            'HH_s':  [np.sin(2 * np.pi * h / 24)],
            'HH_c':  [np.cos(2 * np.pi * h / 24)],
            'Sunspot_No': [sunspot],
            'Lat': [38.0],
            'Lon': [9.0],
            'f10_7_index': [f107]
        })
        pred = lgb_model.predict(scaler.transform(row_df[FEATURES]))[0]
        preds.append(pred)

    # Updated graph layout
    fig = go.Figure()
    fig.add_trace(go.Scatter(
        x=hours, y=preds, mode='lines+markers',
        line=dict(color='#cccccc', width=3),
        marker=dict(size=8, color='#00aaff'),
        name='Predicted vTEC'
    ))

    fig.update_layout(
        title=f"Predicted Daily vTEC for {date.strftime('%Y-%m-%d')} (Addis Ababa)",
        xaxis_title="Hour (UTC)",
        yaxis_title="vTEC (TECU)",
        template="plotly_dark",
        plot_bgcolor="#001f3f",
        paper_bgcolor="rgba(0,0,0,0)",
        font=dict(color="green", size=16),
        xaxis=dict(showgrid=True, gridcolor="rgba(100,100,100,0.3)", tickfont=dict(size=12)),
        yaxis=dict(showgrid=True, gridcolor="rgba(100,100,100,0.3)", tickfont=dict(size=12)),
        hovermode="x unified"
    )

    note = f"☀ NOAA SSN = {sunspot:.1f}, F10.7 = {f107:.1f} for {date.strftime('%Y-%m-%d')}"
    return fig, note

# ==========================================================
# 4️⃣ RUN APP — LOCAL + RENDER FRIENDLY
# ==========================================================
if __name__ == "__main__":
    port = int(os.environ.get("PORT", 8060))
    print("\n==========================================================")
    print("🌍 Project: Total Electron Content (vTEC) Prediction — Addis Ababa")
    print("🏛 Organization: Ethiopian Space Science and Geospatial Institute (ESSGI)")
    print("🎓 Internship by: Debre Birhan University Data Science Students")
    print("👨‍💻 Developers: Haile Sintayehu & Esayas Melaku")
    print(f"🚀 App running on http://127.0.0.1:{port}/")
    print("==========================================================\n")
    app.run(host="0.0.0.0", port=port, debug=True)


[LightGBM] [Info] Start training from score 21.906338183200916
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.007145 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 982
[LightGBM] [Info] Number of data points in the train set: 49967, number of used features: 8
[LightGBM] [Info] Start training from score 21.906338

🌍 Project: Total Electron Content (vTEC) Prediction — Addis Ababa
🏛 Organization: Ethiopian Space Science and Geospatial Institute (ESSGI)
🎓 Internship by: Debre Birhan University Data Science Students
👨‍💻 Developers: Haile Sintayehu & Esayas Melaku
🚀 App running on http://127.0.0.1:8060/



ConnectionError: HTTPConnectionPool(host='0.0.0.0', port=8060): Max retries exceeded with url: /_alive_53a58693-4aae-4464-9ffe-b8f7e4ed6296 (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x000002647E031400>: Failed to establish a new connection: [WinError 10049] The requested address is not valid in its context'))

In [48]:
###############################################################################################################